In [ ]:
!pip install pyts
!pip install --upgrade pyts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from scipy import stats
from scipy.fft import fft, dct
import pywt
from matplotlib.backends.backend_pdf import PdfPages
import os
from pyts.approximation import PiecewiseAggregateApproximation as PAA
from scipy.fftpack import idct

def generate_random_vectors(n, r):
    random_vectors = np.random.normal(0, 1, size=(r, n))
    return random_vectors

def MINE(data,R,R_inv):

  prod = np.matmul(data,R)
  bitmap = np.where(prod < 0, 0, 1)
  recon = np.matmul(bitmap,R_inv)
  return recon

def plot_confidence_interval(scores,method,stride,compression,type):
    # compute mean silhouette score and confidence interval
    mean_score = np.mean(scores)
    ci = stats.t.interval(0.95, len(scores)-1, loc=mean_score, scale=stats.sem(scores))

    # 2D array with the lower and upper bounds of the confidence interval
    yerr = np.array([[mean_score-ci[0]], [ci[1]-mean_score]])

    # create the figure and axis objects
    fig, ax = plt.subplots()

    # plot the data with error bars
    ax.plot(scores, marker='o')
    ax.errorbar(x=range(len(scores)), y=scores, yerr=yerr, fmt='none', ecolor='r')

    # set the axis labels and title
    ax.set_xlabel('Subset of features')

    if type == 'SC' :
     ax.set_ylabel('R2 score')
     if method == 'RAW':
      ax.set_title(f'R2-scores with 95% confidence interval\n M={method} , W= {stride} : [{ci[0]:.5f}, {ci[1]:.5f}]')
     else:
      ax.set_title(f'R2-scores with 95% confidence interval\n M={method} , W= {stride}, C= {compression} : [{ci[0]:.5f}, {ci[1]:.5f}]')


    if type == 'RMSE' :
     ax.set_ylabel('RMSE-score')
     if method == 'RAW':
      ax.set_title(f'RMSE-scores with 95% confidence interval\n M={method} , W= {stride} : [{ci[0]:.5f}, {ci[1]:.5f}]')
     else:
      ax.set_title(f'RMSE-scores with 95% confidence interval\n M={method} , W= {stride}, C= {compression} : [{ci[0]:.5f}, {ci[1]:.5f}]')
    # return the figure object
    return fig


def plot_score_confidence_interval(data,window_size,compression):
    fig, axs = plt.subplots(1, 5, figsize=(20, 4))  # Adjust the figure size as needed

    colors = ['b', 'g', 'r', 'c', 'm']  # Colors for each method

    y_axis_limits = (-1.5, 1.5)  # Outlier limits

    for i, (scores, method) in enumerate(data):
        ax = axs[i]
        n = len(scores)
        x = np.arange(n) + 1
        y = np.array(scores)

        # Identify and count outliers based on the limits
        outliers = np.where((y < -1.5) | (y > 1.5))[0]
        num_outliers = len(outliers)

        # Exclude outliers from the data
        filtered_scores = np.delete(y, outliers)
        filtered_x = np.delete(x, outliers)

        mean = np.mean(filtered_scores)
        # Calculate the confidence interval
        ci = stats.t.interval(0.95, len(filtered_scores), loc=mean, scale=stats.sem(filtered_scores))
        # Plot the non-outlier data points and mean
        ax.plot(filtered_x, filtered_scores, 'o', markersize=4, color=colors[i])
        ax.plot(x, [mean] * n, '--', color=colors[i])

        fig.suptitle(f'R2-scores with 95% confidence interval // W= {window_size}, C= {compression} ')

        ax.set_xlabel('Window')
        ax.set_ylabel('Score')
        ax.set_title(f'{method} : [{ci[0]:.5f}, {ci[1]:.5f}] ')
        ax.grid(True)

        # Set the y-axis limits to the predefined range
        ax.set_ylim(y_axis_limits)

        # Add the number of outliers as text in the top right corner
        ax.text(0.85, 0.9, f'Outliers: {num_outliers}', transform=ax.transAxes, fontsize=10, color='red', ha='right')

    return fig


def plot_RMSE_confidence_interval(data, window_size, compression, outlier_threshold=3):
    fig, axs = plt.subplots(1, 5, figsize=(20, 4))  # Adjust the figure size as needed

    colors = ['b', 'g', 'r', 'c', 'm']  # Colors for each method

    y_axis_limits = (-1.5, 1.5) # Outlier limits

    for i, (scores, method) in enumerate(data):
        ax = axs[i]
        n = len(scores)
        x = np.arange(n) + 1
        y = np.array(scores)
        # Identify and count outliers based on the limits
        outliers = np.where((y < -1.5) | (y > 1.5))[0]
        num_outliers = len(outliers)

        # Exclude outliers from the data
        filtered_scores = np.delete(y, outliers)
        filtered_x = np.delete(x, outliers)

        mean = np.mean(filtered_scores)
        # Calculate the confidence interval
        ci = stats.t.interval(0.95, len(filtered_scores), loc=mean, scale=stats.sem(filtered_scores))

        # Plot the non-outlier data points and mean
        ax.plot(filtered_x, filtered_scores, 'o', markersize=4, color=colors[i])
        ax.plot(x, [mean] * n, '--', color=colors[i])

        fig.suptitle(f'RMSE-scores with 95% confidence interval // W= {window_size}, C= {compression} ')

        ax.set_xlabel('Window')
        ax.set_ylabel('Score')
        ax.set_title(f'{method} : [{0}, {ci[1]:.5f}]')
        ax.grid(True)

        # Set the y-axis limits to the predefined range
        ax.set_ylim(y_axis_limits)

        # Add the number of outliers as text in the top right corner
        ax.text(0.85, 0.9, f'Outliers: {num_outliers}', transform=ax.transAxes, fontsize=10, color='red', ha='right')

    return fig


# Load data from file
all_temperatures = np.loadtxt("/content/drive/MyDrive/temp.txt", delimiter='\t')
all_humidities = np.loadtxt("/content/drive/MyDrive/humidity.txt", delimiter='\t')
all_lights = np.loadtxt("/content/drive/MyDrive/light.txt", delimiter='\t')
all_voltages = np.loadtxt("/content/drive/MyDrive/voltage.txt", delimiter='\t')

alldata = [all_temperatures.T , all_humidities.T , all_lights.T , all_voltages.T ]


count = 0
for data in alldata:

  # window size
  window_sizes = [16, 32, 64, 128]
  compressions = [4, 8, 16]
  methods = ['DFT', 'DCT', 'DWT', 'PAA','MINE']
  if count == 0:
    dstring = 'Temperatures'
  elif count == 1:
     dstring = 'Humidities'
  elif count == 2:
     dstring = 'Lights'
  elif count == 3:
    dstring = 'Voltages'
  count = count + 1
  #file for storing
  folder_name = f"Linear Regression {dstring}  Measurements"

  if not os.path.exists(folder_name):
    os.makedirs(folder_name)

  raw_folder_name = f"RAW DATA"
  if not os.path.exists(os.path.join(folder_name,raw_folder_name)):
    os.makedirs(os.path.join(folder_name,raw_folder_name))
  pdf_file = os.path.join(folder_name,raw_folder_name, 'plots.pdf')

  with PdfPages(pdf_file, 'a') as pdf:
    for window_size in window_sizes:

      # lists to store metrics for each window
      rmse_list = []
      #mae_list = []
      r2_list = []
      num =int( window_size+window_size/4)

      for i in range(0, data.shape[1]-window_size*2+1, window_size):
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(data[:, i:i+window_size], data[:, i+window_size:i+num], test_size=0.25, random_state=13)

            #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

            # Data scaling
            sc = MinMaxScaler()
            X_train_scaled = sc.fit_transform(X_train)
            X_test_scaled = sc.transform(X_test)
            y_train_scaled = sc.fit_transform(y_train)
            y_test_scaled = sc.transform(y_test)

            # Model application
            regressor = LinearRegression()
            regressor.fit(X_train_scaled, y_train_scaled)

            # Generate predictions
            y_pred_scaled = regressor.predict(X_test_scaled)

            # performance (mean squared error, mean absolute error, and R-squared score)
            mse = mean_squared_error(y_test_scaled, y_pred_scaled)
            rmse = np.sqrt(mse)
            #mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
            r2 = r2_score(y_test_scaled, y_pred_scaled)
            rmse_list.append(rmse)
            r2_list.append(r2)

      #store plots
      methd = 'RAW'
      comprsn = 0
      fig = plot_confidence_interval(r2_list,methd,window_size ,comprsn,'SC') #plot function call
      pdf.savefig(fig)
      plt.close(fig)

      fig2 = plot_confidence_interval(rmse_list,methd,window_size ,comprsn,'RMSE') #plot function call
      pdf.savefig(fig2)
      plt.close(fig2)

      # create a file name based on the method, subset index, stride, and compression
      avg_r2 = np.mean(r2_list)
      file_name = f"Window size({window_size}) - R2-scores.txt"
      with open(os.path.join(folder_name,raw_folder_name, file_name), 'w+') as f:
                f.write(f"Average R2 Score: {avg_r2}\n")
                content = str(r2_list)
                f.write(content)
                f.close()

      avg_rmse = np.mean(rmse_list)
      file_name = f"Window size({window_size}) - RMSE scores).txt"
      with open(os.path.join(folder_name,raw_folder_name, file_name), 'w+') as f:
                f.write(f"Average RMSE Score: {avg_rmse}\n")
                content = str(rmse_list)
                f.write(content)
                f.close()


  for window_size in window_sizes:

      #file for storing
      stride_folder_name = f"stride({window_size})"
      if not os.path.exists(os.path.join(folder_name,stride_folder_name)):
          os.makedirs(os.path.join(folder_name,stride_folder_name))

      for compression in compressions:

          # #vector and inverse vector arrays for our algorithm
          n = 64*int(window_size/compression)  # Dimension of each random vector
          r = window_size  # Number of random vectors to generate/rows
          R = generate_random_vectors(n, r)
          R_inv= np.linalg.pinv(R)
        # print(R.shape,R_inv.shape)

          #files for storing measurements
          compression_folder_name = f"compression({compression})"
          if not os.path.exists(os.path.join(folder_name,stride_folder_name, compression_folder_name)):
              os.makedirs(os.path.join(folder_name,stride_folder_name, compression_folder_name))

          #files for storing plots
          plots_dir = os.path.join(folder_name,stride_folder_name, compression_folder_name, 'plots')
          os.makedirs(plots_dir, exist_ok=True)
          pdf_file = os.path.join(plots_dir, 'plots.pdf')

          with PdfPages(pdf_file, 'a') as pdf:
              results1 = []
              results2 = []
              for method in methods:

                  slices = window_size // compression

                  # lists to store metrics for each window
                  rmse_list = []
                  #mae_list = []
                  r2_list = []
                  num =int( window_size+window_size/4)

                  # Iterate over the windows
                  for i in range(0, data.shape[1]-window_size*2+1, window_size):
                      #vector and inverse vector arrays for our algorithm
                      # n = 64*int(window_size/compression)  # Dimension of each random vector
                      # r = window_size  # Number of random vectors to generate/rows
                      # R = generate_random_vectors(n, r)
                      # R_inv= np.linalg.pinv(R)

                      # Split data
                      X_train, X_test, y_train, y_test = train_test_split(data[:, i:i+window_size], data[:, i+window_size:i+num], test_size=0.25, random_state=13)
                      #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

                      # Data scaling
                      sc = MinMaxScaler()
                      X_train_scaled = sc.fit_transform(X_train)
                      X_test_scaled = sc.transform(X_test)
                      y_train_scaled = sc.fit_transform(y_train)
                      y_test_scaled = sc.transform(y_test)

                      if method == 'DFT':

                        #array to store the restored data
                        compressed_X_train_scaled = np.zeros_like(X_train_scaled)

                        for i in range(36):
                            # Compute abs DFT of each row
                            dft_X_train_scaled  =  np.fft.fft(X_train_scaled[i])
                            # Sort the DFT coefficients along each row by their magnitude
                            sorted_indices = np.argsort(-np.abs(dft_X_train_scaled))[:slices]
                            sorted_dft_X_train_scaled = dft_X_train_scaled[sorted_indices]
                            # Keep  top  coeff
                            top__dft_X_train_scaled = sorted_dft_X_train_scaled[:slices]
                            #reconstruct the compressed dataset
                            compressed_X_train_scaled[i] = np.fft.ifft(top__dft_X_train_scaled,window_size).real

                            # print('be4 dft',X_train_scaled[i].shape,X_train_scaled[i])
                            # print('after dft', dft_X_train_scaled .shape,dft_X_train_scaled )
                            # print('top dft ',top__dft_X_train_scaled.shape,top__dft_X_train_scaled)
                            # print('after',compressed_X_train_scaled.shape,compressed_X_train_scaled)
                            # dfhdfgh

                        #array to store the restored data
                        compressed_X_test_scaled = np.zeros_like(X_test_scaled)

                        for i in range(12):
                            # Compute abs DFT of each row
                            dft_X_test_scaled =  np.fft.fft(X_test_scaled[i])
                            # Sort the DFT coefficients along each row by their magnitude
                            sorted_indices = np.argsort(-np.abs(dft_X_test_scaled))[:slices]
                            sorted_dft_X_test_scaled = dft_X_test_scaled[sorted_indices]
                            # Keep  top  coeff
                            top__dft_X_test_scaled = sorted_dft_X_test_scaled[:slices]
                            #reconstruct the compressed dataset
                            compressed_X_test_scaled[i] = np.fft.ifft(top__dft_X_test_scaled,window_size).real

                      elif method == 'DCT':

                        #array to store the restored data
                        compressed_X_train_scaled = np.zeros_like(X_train_scaled)

                        for i in range(36):
                            # Compute abs DCT of each row
                            dct_X_train_scaled  =  dct(X_train_scaled[i])
                            # Sort the DCT coefficients along each row by their magnitude
                            sorted_indices = np.argsort(-np.abs(dct_X_train_scaled))[:slices]
                            sorted_dct_X_train_scaled = dct_X_train_scaled[sorted_indices]
                            # Keep  top  coeff
                            top__dct_X_train_scaled = sorted_dct_X_train_scaled[:slices]
                            #reconstruct the compressed dataset
                            compressed_X_train_scaled[i] = idct(top__dct_X_train_scaled ,type = 2, n = window_size).real

                        #array to store the restored data
                        compressed_X_test_scaled = np.zeros_like(X_test_scaled)

                        for i in range(12):
                            # Compute abs DCT of each row
                            dct_X_test_scaled =  dct(X_test_scaled[i])
                            # Sort the DCT coefficients along each row by their magnitude
                            sorted_indices = np.argsort(-np.abs(dct_X_test_scaled))[:slices]
                            sorted_dct_X_test_scaled = dct_X_test_scaled[sorted_indices]
                            # Keep  top  coeff
                            top__dct_X_test_scaled = sorted_dct_X_test_scaled[:slices]
                            #reconstruct the compressed dataset
                            compressed_X_test_scaled[i] = idct(top__dct_X_test_scaled ,type = 2 , n = window_size).real


                      elif method == 'DWT':

                        #array to store the restored data
                        compressed_X_train_scaled = np.zeros_like(X_train_scaled)

                        for i in range(36):
                            # Apply DWT to each row using 'db1' wavelet
                            cA, cD = pywt.dwt(X_train_scaled[i], 'db1')
                            # Sort
                            sorted_cD_subset_data = np.zeros_like(cD) #initialize storage array
                            sorted_indices = np.argsort(-np.abs(cD))[:slices]
                            sorted_cD_subset_data[sorted_indices] = cD[sorted_indices]
                            # Perform inverse DWT to restore the row
                            compressed_X_train_scaled[i] = pywt.idwt(cA, sorted_cD_subset_data, 'db1')

                        #array to store the restored data
                        compressed_X_test_scaled = np.zeros_like(X_test_scaled)

                        for i in range(12):
                            # Apply DWT to each row using 'db1' wavelet
                            cA, cD = pywt.dwt(X_test_scaled[i], 'db1')
                            # Sort
                            sorted_cD_subset_data = np.zeros_like(cD) #initialize storage array
                            sorted_indices = np.argsort(-np.abs(cD))[:slices]
                            sorted_cD_subset_data[sorted_indices] = cD[sorted_indices]
                            # Perform inverse DWT to restore the row
                            compressed_X_test_scaled[i] = pywt.idwt(cA, sorted_cD_subset_data, 'db1')

                      elif method == 'PAA':
                        # Apply PAA along the rows of the array
                        paa = PAA(window_size = compression)
                        compressed_X_train_scaled = paa.fit_transform(X_train_scaled)
                        compressed_X_test_scaled = paa.fit_transform(X_test_scaled)

                      elif method == 'MINE':

                        #array to store the restored data
                        compressed_X_train_scaled = np.zeros_like(X_train_scaled)

                        for i in range(36):
                          row = X_train_scaled[i]
                          # Apply my algorithm along the rows of the array
                          compressed_X_train_scaled[i] = MINE(row,R,R_inv)

                        #array to store the restored data
                        compressed_X_test_scaled = np.zeros_like(X_test_scaled)
                        for i in range(12):
                          row = X_test_scaled[i]
                          # Apply my algorithm along the rows of the array
                          compressed_X_test_scaled[i] = MINE(row,R,R_inv)

                      # Model application
                      regressor = LinearRegression()
                      regressor.fit(compressed_X_train_scaled, y_train_scaled)

                      # Generate predictions
                      y_pred_scaled = regressor.predict(compressed_X_test_scaled)

                      # Transform scaled predictions back to og scale
                      #y_pred = sc.inverse_transform(y_pred_scaled)

                      # performance (mean squared error, mean absolute error, and R-squared score)
                      mse = mean_squared_error(y_test_scaled, y_pred_scaled)
                      rmse = np.sqrt(mse)
                      #mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
                      r2 = r2_score(y_test_scaled, y_pred_scaled)

                      # Add to list of measurements
                      rmse_list.append(rmse)
                      #mae_list.append(mae)
                      r2_list.append(r2)

                  results1.append((r2_list, method))
                  results2.append((rmse_list, method))

                  # create a file name based on the method, subset index, stride, and compression
                  avg_r2 = np.mean(r2_list)
                  file_name = f"{method}_window size({window_size})_compression({compression} - R2 SCORES).txt"
                  # write the silhouette coefficients to file
                  with open(os.path.join(folder_name,stride_folder_name, compression_folder_name, file_name), 'w+') as f:
                                f.write(f"Average R2 Score: {avg_r2}\n")
                                content = str(r2_list)
                                f.write(content)
                                f.close()

                  avg_RMSE = np.mean(rmse_list)
                  file_name = f"{method}_window size({window_size})_compression({compression} - RMSEs).txt"
                  # write the silhouette coefficients to file
                  with open(os.path.join(folder_name,stride_folder_name, compression_folder_name, file_name), 'w+') as f:
                                f.write(f"Average RMSE Score: {avg_RMSE}\n")
                                content = str(rmse_list)
                                f.write(content)
                                f.close()
              #store plots
              fig = plot_score_confidence_interval(results1,window_size,compression) #plot function call
              pdf.savefig(fig)
              plt.close(fig)
              fig = plot_RMSE_confidence_interval(results2,window_size,compression) #plot function call
              pdf.savefig(fig)
              plt.close(fig)




/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=a

In [ ]:
#@title MAIN BACKUP
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from scipy import stats
from scipy.fft import fft, dct
import pywt
from matplotlib.backends.backend_pdf import PdfPages
import os
from pyts.approximation import PiecewiseAggregateApproximation as PAA
from scipy.fftpack import idct



def plot_confidence_interval(scores,method,stride,compression,type):
    # compute mean silhouette score and confidence interval
    mean_score = np.mean(scores)
    ci = stats.t.interval(0.95, len(scores)-1, loc=mean_score, scale=stats.sem(scores))

    # 2D array with the lower and upper bounds of the confidence interval
    yerr = np.array([[mean_score-ci[0]], [ci[1]-mean_score]])

    # create the figure and axis objects
    fig, ax = plt.subplots()

    # plot the data with error bars
    ax.plot(scores, marker='o')
    ax.errorbar(x=range(len(scores)), y=scores, yerr=yerr, fmt='none', ecolor='r')

    # set the axis labels and title
    ax.set_xlabel('Subset of features')

    if type == 'SC' :
     ax.set_ylabel('R2 score')
     if method == 'RAW':
      ax.set_title(f'R2 scores with 95% confidence interval\n M={method} , W= {stride} : [{ci[0]:.5f}, {ci[1]:.5f}]')
     else:
      ax.set_title(f'R2 scores with 95% confidence interval\n M={method} , W= {stride}, C= {compression} : [{ci[0]:.5f}, {ci[1]:.5f}]')


    if type == 'RMSE' :
     ax.set_ylabel('RMSE score')
     if method == 'RAW':
      ax.set_title(f'RMSE scores with 95% confidence interval\n M={method} , W= {stride} : [{ci[0]:.5f}, {ci[1]:.5f}]')
     else:
      ax.set_title(f'RMSE scores with 95% confidence interval\n M={method} , W= {stride}, C= {compression} : [{ci[0]:.5f}, {ci[1]:.5f}]')
    # return the figure object
    return fig


# Load data from file
all_temperatures = np.loadtxt("/content/drive/MyDrive/temp.txt", delimiter='\t')
all_humidities = np.loadtxt("/content/drive/MyDrive/humidity.txt", delimiter='\t')
all_lights = np.loadtxt("/content/drive/MyDrive/light.txt", delimiter='\t')
all_voltages = np.loadtxt("/content/drive/MyDrive/voltage.txt", delimiter='\t')

data = all_temperatures.T

# window size
window_sizes = [16, 32, 64, 128]
compressions = [4, 8, 16]
methods = ['DFT', 'DCT', 'DWT', 'PAA']

#file for storing
folder_name = f"Linear Regression temperatures Measurements"
if not os.path.exists(folder_name):
   os.makedirs(folder_name)

raw_folder_name = f"RAW DATA"
if not os.path.exists(os.path.join(folder_name,raw_folder_name)):
   os.makedirs(os.path.join(folder_name,raw_folder_name))
pdf_file = os.path.join(folder_name,raw_folder_name, 'plots.pdf')

with PdfPages(pdf_file, 'a') as pdf:
  for window_size in window_sizes:

     # lists to store metrics for each window
     rmse_list = []
     #mae_list = []
     r2_list = []
     num =int( window_size+window_size/4)

     for i in range(0, data.shape[1]-window_size*2+1, window_size):
          # Split data
          X_train, X_test, y_train, y_test = train_test_split(data[:, i:i+window_size], data[:, i+window_size:i+num], test_size=0.25, random_state=13)

          #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

          # Data scaling
          sc = MinMaxScaler()
          X_train_scaled = sc.fit_transform(X_train)
          X_test_scaled = sc.transform(X_test)
          y_train_scaled = sc.fit_transform(y_train)
          y_test_scaled = sc.transform(y_test)

          # Model application
          regressor = LinearRegression()
          regressor.fit(X_train_scaled, y_train_scaled)

          # Generate predictions
          y_pred_scaled = regressor.predict(X_test_scaled)

          # performance (mean squared error, mean absolute error, and R-squared score)
          mse = mean_squared_error(y_test_scaled, y_pred_scaled)
          rmse = np.sqrt(mse)
          #mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
          r2 = r2_score(y_test_scaled, y_pred_scaled)
          rmse_list.append(rmse)
          r2_list.append(r2)

     #store plots
     methd = 'RAW'
     comprsn = 0
     fig = plot_confidence_interval(r2_list,methd,window_size ,comprsn,'SC') #plot function call
     pdf.savefig(fig)
     plt.close(fig)

     fig2 = plot_confidence_interval(rmse_list,methd,window_size ,comprsn,'RMSE') #plot function call
     pdf.savefig(fig2)
     plt.close(fig2)

     # create a file name based on the method, subset index, stride, and compression
     avg_r2 = np.mean(r2_list)
     file_name = f"Window size({window_size}) - R2-scores.txt"
     with open(os.path.join(folder_name,raw_folder_name, file_name), 'w+') as f:
              f.write(f"Average R2 Score: {avg_r2}\n")
              content = str(r2_list)
              f.write(content)
              f.close()

     avg_rmse = np.mean(rmse_list)
     file_name = f"Window size({window_size}) - RMSE scores).txt"
     with open(os.path.join(folder_name,raw_folder_name, file_name), 'w+') as f:
              f.write(f"Average RMSE Score: {avg_rmse}\n")
              content = str(rmse_list)
              f.write(content)
              f.close()


for window_size in window_sizes:

    #file for storing
    stride_folder_name = f"stride({window_size})"
    if not os.path.exists(os.path.join(folder_name,stride_folder_name)):
        os.makedirs(os.path.join(folder_name,stride_folder_name))

    for compression in compressions:

        #files for storing measurements
        compression_folder_name = f"compression({compression})"
        if not os.path.exists(os.path.join(folder_name,stride_folder_name, compression_folder_name)):
            os.makedirs(os.path.join(folder_name,stride_folder_name, compression_folder_name))

        #files for storing plots
        plots_dir = os.path.join(folder_name,stride_folder_name, compression_folder_name, 'plots')
        os.makedirs(plots_dir, exist_ok=True)
        pdf_file = os.path.join(plots_dir, 'plots.pdf')

        with PdfPages(pdf_file, 'a') as pdf:
            for method in methods:

                slices = window_size // compression

                # lists to store metrics for each window
                rmse_list = []
                #mae_list = []
                r2_list = []
                num =int( window_size+window_size/4)
                print(data.shape[1])
                # Iterate over the windows
                for i in range(0, data.shape[1]-window_size*2+1, window_size):

                    # Split data
                    X_train, X_test, y_train, y_test = train_test_split(data[:, i:i+window_size], data[:, i+window_size:i+num], test_size=0.25, random_state=13)
                    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

                    # Data scaling
                    sc = MinMaxScaler()
                    X_train_scaled = sc.fit_transform(X_train)
                    X_test_scaled = sc.transform(X_test)
                    y_train_scaled = sc.fit_transform(y_train)
                    y_test_scaled = sc.transform(y_test)

                    if method == 'DFT':

                       # Compute abs DFT of each row of X_train
                       dft_X_train_scaled = np.fft.fft(X_train_scaled, axis=1)
                       abs_dft_X_train_scaled = np.abs(dft_X_train_scaled)
                       # Sort ROWS
                       sorted_dft_X_train_scaled = np.sort(abs_dft_X_train_scaled, axis=1)[:, ::-1]
                       # Keep  top  coeff
                       top4_dft_X_train_scaled = sorted_dft_X_train_scaled[:, :slices]
                       #reconstruct the compressed dataset
                       compressed_X_train_scaled = np.fft.ifft(top4_dft_X_train_scaled, axis=1).real

                       # Compute abs DFT of each row of X_test
                       dft_X_test_scaled = np.fft.fft(X_test_scaled, axis=1)
                       abs_dft_X_test_scaled = np.abs(dft_X_test_scaled)
                       # Sort ROWS
                       sorted_dft_X_test_scaled = np.sort(abs_dft_X_test_scaled, axis=1)[:, ::-1]
                       # Keep  top  coeff
                       top4_dft_X_test_scaled = sorted_dft_X_test_scaled[:, :slices]
                       #reconstruct the compressed dataset
                       compressed_X_test_scaled = np.fft.ifft(top4_dft_X_test_scaled, axis=1).real

                    elif method == 'DCT':

                       # Compute DCT of each row of X_train
                       dct_X_train_scaled = dct(X_train_scaled, axis=1)
                       # Sort ROWS
                       sorted_dct_X_train_scaled = np.sort(np.abs(dct_X_train_scaled), axis=1)[:, ::-1]
                       # Keep  top  coeff
                       top4_dct_X_train_scaled = sorted_dct_X_train_scaled[:, :slices]
                       #reconstruct the compressed dataset
                       compressed_X_train_scaled = idct(top4_dct_X_train_scaled, axis=1).real

                       # Compute abs DFT of each row of X_test
                       dct_X_test_scaled = dct(X_test_scaled, axis=1)
                       # Sort ROWS
                       sorted_dct_X_test_scaled = np.sort(np.abs(dct_X_test_scaled), axis=1)[:, ::-1]
                       # Keep  top  coeff
                       top4_dct_X_test_scaled = sorted_dct_X_test_scaled[:, :slices]
                       #reconstruct the compressed dataset
                       compressed_X_test_scaled = idct(top4_dct_X_test_scaled, axis=1).real

                      #  print('be4',X_train_scaled[0],X_train_scaled[1])
                      #  print('dct',dct_X_train_scaled.shape,dct_X_train_scaled[0],dct_X_train_scaled[1])
                      #  print('comped',top4_dct_X_train_scaled[0], top4_dct_X_train_scaled[1])
                      #  print('reconed',compressed_X_train_scaled.shape,compressed_X_train_scaled[0],compressed_X_train_scaled[1])

                    elif method == 'DWT':

                      # Apply DWT to X_train_scaled
                      cA, cD = pywt.dwt(X_train_scaled, 'db1', axis=1)
                      # Sort ROWS
                      sorted_cA = np.sort(np.abs(cA), axis=1)[:, ::-1]
                      # select biggest coefficients
                      X_train_compressed = sorted_cA[:, :slices]
                      # Reconstruct original data
                      compressed_X_train_scaled = pywt.idwt(X_train_compressed, None, 'db1')

                      # Apply DWT to X_train_scaled
                      cA, cD = pywt.dwt(X_test_scaled, 'db1', axis=1)
                      # Sort ROWS
                      sorted_cA = np.sort(np.abs(cA), axis=1)[:, ::-1]
                      # select biggest coefficients
                      X_test_compressed = sorted_cA[:, :slices]
                      # Reconstruct original data
                      compressed_X_test_scaled = pywt.idwt(X_test_compressed, None, 'db1')


                    elif method == 'PAA':
                      # Apply PAA along the rows of the array
                      paa = PAA(window_size = compression)
                      compressed_X_train_scaled = paa.fit_transform(X_train_scaled)
                      compressed_X_test_scaled = paa.fit_transform(X_test_scaled)


                    # Model application
                    regressor = LinearRegression()
                    regressor.fit(compressed_X_train_scaled, y_train_scaled)

                    # Generate predictions
                    y_pred_scaled = regressor.predict(compressed_X_test_scaled)

                    # Transform scaled predictions back to og scale
                    #y_pred = sc.inverse_transform(y_pred_scaled)

                    # performance (mean squared error, mean absolute error, and R-squared score)
                    mse = mean_squared_error(y_test_scaled, y_pred_scaled)
                    rmse = np.sqrt(mse)
                    #mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
                    r2 = r2_score(y_test_scaled, y_pred_scaled)

                    # Add to list of measurements
                    rmse_list.append(rmse)
                    #mae_list.append(mae)
                    r2_list.append(r2)

                #store plots
                fig = plot_confidence_interval(r2_list,method,window_size,compression,'SC') #plot function call
                pdf.savefig(fig)
                plt.close(fig)
                fig2 = plot_confidence_interval(rmse_list,method,window_size ,compression,'RMSE') #plot function call
                pdf.savefig(fig2)
                plt.close(fig2)

                # create a file name based on the method, subset index, stride, and compression
                avg_r2 = np.mean(r2_list)
                file_name = f"{method}_window size({window_size})_compression({compression} - R2 SCORES).txt"
                # write the silhouette coefficients to file
                with open(os.path.join(folder_name,stride_folder_name, compression_folder_name, file_name), 'w+') as f:
                              f.write(f"Average R2 Score: {avg_r2}\n")
                              content = str(r2_list)
                              f.write(content)
                              f.close()

                avg_RMSE = np.mean(rmse_list)
                file_name = f"{method}_window size({window_size})_compression({compression} - RMSEs).txt"
                # write the silhouette coefficients to file
                with open(os.path.join(folder_name,stride_folder_name, compression_folder_name, file_name), 'w+') as f:
                              f.write(f"Average ARI Score: {avg_RMSE}\n")
                              content = str(rmse_list)
                              f.write(content)
                              f.close()





In [ ]:
#@title Backup
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from scipy import stats
from scipy.fft import fft, dct
import pywt
from matplotlib.backends.backend_pdf import PdfPages
import os

def plot_confidence_interval(scores,method,stride,compression,type):
    # compute mean silhouette score and confidence interval
    mean_score = np.mean(scores)
    ci = stats.t.interval(0.95, len(scores)-1, loc=mean_score, scale=stats.sem(scores))

    # 2D array with the lower and upper bounds of the confidence interval
    yerr = np.array([[mean_score-ci[0]], [ci[1]-mean_score]])

    # create the figure and axis objects
    fig, ax = plt.subplots()

    # plot the data with error bars
    ax.plot(scores, marker='o')
    ax.errorbar(x=range(len(scores)), y=scores, yerr=yerr, fmt='none', ecolor='r')

    # set the axis labels and title
    ax.set_xlabel('Subset of features')

    if type == 'SC' :
     ax.set_ylabel('R2 score')
     if method == 'RAW':
      ax.set_title(f'R2 scores with 95% confidence interval\n M={method} , W= {stride} : [{ci[0]:.5f}, {ci[1]:.5f}]')
     else:
      ax.set_title(f'R2 scores with 95% confidence interval\n M={method} , W= {stride}, C= {compression} : [{ci[0]:.5f}, {ci[1]:.5f}]')


    if type == 'RMSE' :
     ax.set_ylabel('RMSE score')
     if method == 'RAW':
      ax.set_title(f'RMSE scores with 95% confidence interval\n M={method} , W= {stride} : [{ci[0]:.5f}, {ci[1]:.5f}]')
     else:
      ax.set_title(f'RMSE scores with 95% confidence interval\n M={method} , W= {stride}, C= {compression} : [{ci[0]:.5f}, {ci[1]:.5f}]')
    # return the figure object
    return fig

def paa(data, n_pieces):
    """
    Piecewise Aggregate Approximation (PAA) on data.
    """
    n = data.shape[0]
    piece_length = int(np.ceil(n/n_pieces))
    padded_data = np.pad(data, ((0, piece_length*n_pieces-n), (0,0)), mode='constant', constant_values=0)
    pieces = padded_data.reshape(n_pieces, piece_length, -1)
    return np.mean(pieces, axis=1)


# Load data from file
all_temperatures = np.loadtxt("/content/drive/MyDrive/temp.txt", delimiter='\t')
all_humidities = np.loadtxt("/content/drive/MyDrive/humidity.txt", delimiter='\t')
all_lights = np.loadtxt("/content/drive/MyDrive/light.txt", delimiter='\t')
all_voltages = np.loadtxt("/content/drive/MyDrive/voltage.txt", delimiter='\t')

data = all_voltages.T

# window size
window_sizes = [16, 32, 64, 128]
compressions = [4, 8, 16]
methods = ['DFT', 'DCT', 'DWT', 'PAA']

#file for storing
folder_name = f"Linear Regression voltages  Measurements"
if not os.path.exists(folder_name):
   os.makedirs(folder_name)

raw_folder_name = f"RAW DATA"
if not os.path.exists(os.path.join(folder_name,raw_folder_name)):
   os.makedirs(os.path.join(folder_name,raw_folder_name))
pdf_file = os.path.join(folder_name,raw_folder_name, 'plots.pdf')

with PdfPages(pdf_file, 'a') as pdf:
  for window_size in window_sizes:

     # lists to store metrics for each window
     rmse_list = []
     #mae_list = []
     r2_list = []

     for i in range(0, data.shape[1]-window_size*2+1, window_size):
          # Split data
          X_train, X_test, y_train, y_test = train_test_split(data[:, i:i+window_size], data[:, i+window_size:i+window_size*2], test_size=0.25, random_state=13)
          #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

          # Data scaling
          sc = MinMaxScaler()
          X_train_scaled = sc.fit_transform(X_train)
          X_test_scaled = sc.transform(X_test)
          y_train_scaled = sc.fit_transform(y_train.reshape(y_train.shape[0],-1))
          y_test_scaled = sc.transform(y_test.reshape(y_test.shape[0],-1))

          # Model application
          regressor = LinearRegression()
          regressor.fit(X_train_scaled, y_train_scaled)

          # Generate predictions
          y_pred_scaled = regressor.predict(X_test_scaled)

          # performance (mean squared error, mean absolute error, and R-squared score)
          mse = mean_squared_error(y_test_scaled, y_pred_scaled)
          rmse = np.sqrt(mse)
          #mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
          r2 = r2_score(y_test_scaled, y_pred_scaled)
          rmse_list.append(rmse)
          r2_list.append(r2)

     #store plots
     methd = 'RAW'
     comprsn = 0
     fig = plot_confidence_interval(r2_list,methd,window_size ,comprsn,'SC') #plot function call
     pdf.savefig(fig)
     plt.close(fig)

     fig2 = plot_confidence_interval(rmse_list,methd,window_size ,comprsn,'RMSE') #plot function call
     pdf.savefig(fig2)
     plt.close(fig2)

     # create a file name based on the method, subset index, stride, and compression
     avg_r2 = np.mean(r2_list)
     file_name = f"Window size({window_size}) - R2-scores.txt"
     with open(os.path.join(folder_name,raw_folder_name, file_name), 'w+') as f:
              f.write(f"Average R2 Score: {avg_r2}\n")
              content = str(r2_list)
              f.write(content)
              f.close()

     avg_rmse = np.mean(rmse_list)
     file_name = f"Window size({window_size}) - RMSE scores).txt"
     with open(os.path.join(folder_name,raw_folder_name, file_name), 'w+') as f:
              f.write(f"Average RMSE Score: {avg_rmse}\n")
              content = str(rmse_list)
              f.write(content)
              f.close()


for window_size in window_sizes:

    #file for storing
    stride_folder_name = f"stride({window_size})"
    if not os.path.exists(os.path.join(folder_name,stride_folder_name)):
        os.makedirs(os.path.join(folder_name,stride_folder_name))

    for compression in compressions:

        #files for storing measurements
        compression_folder_name = f"compression({compression})"
        if not os.path.exists(os.path.join(folder_name,stride_folder_name, compression_folder_name)):
            os.makedirs(os.path.join(folder_name,stride_folder_name, compression_folder_name))

        #files for storing plots
        plots_dir = os.path.join(folder_name,stride_folder_name, compression_folder_name, 'plots')
        os.makedirs(plots_dir, exist_ok=True)
        pdf_file = os.path.join(plots_dir, 'plots.pdf')

        with PdfPages(pdf_file, 'a') as pdf:
            for method in methods:

                slices = window_size // compression

                # lists to store metrics for each window
                rmse_list = []
                #mae_list = []
                r2_list = []


                # Iterate over the windows
                for i in range(0, data.shape[1]-window_size*2+1, window_size):

                    # Split data
                    X_train, X_test, y_train, y_test = train_test_split(data[:, i:i+window_size], data[:, i+window_size:i+window_size*2], test_size=0.25, random_state=13)
                    #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

                    # Data scaling
                    sc = MinMaxScaler()
                    X_train_scaled = sc.fit_transform(X_train)
                    X_test_scaled = sc.transform(X_test)
                    y_train_scaled = sc.fit_transform(y_train.reshape(y_train.shape[0],-1))
                    y_test_scaled = sc.transform(y_test.reshape(y_test.shape[0],-1))

                    if method == 'DFT':

                       X_train_scaled = np.abs(fft(X_train_scaled.T, axis=0))
                       X_train_scaled  = (np.sort(X_train_scaled , axis=0)[::-1][:slices]).T

                       X_test_scaled = np.abs(fft(X_test_scaled.T, axis=0))
                       X_test_scaled= (np.sort(X_test_scaled, axis=0)[::-1][:slices]).T

                       y_train_scaled = np.abs(fft(y_train_scaled.T, axis=0))
                       y_train_scaled = (np.sort(y_train_scaled, axis=0)[::-1][:slices]).T

                       y_test_scaled = np.abs(fft(y_test_scaled.T, axis=0))
                       y_test_scaled = (np.sort(y_test_scaled, axis=0)[::-1][:slices]).T
                      # print(X_train_scaled .shape, X_test_scaled.shape, y_train_scaled.shape, y_test_scaled.shape)

                    elif method == 'DCT':

                      X_train_scaled = dct(X_train_scaled.T, axis=0)
                      X_test_scaled = dct(X_test_scaled.T, axis=0)
                      y_train_scaled = dct(y_train_scaled.T, axis=0)
                      y_test_scaled = dct(y_test_scaled.T, axis=0)

                      X_train_scaled  = (np.sort(X_train_scaled , axis=0)[::-1][:slices]).T
                      X_test_scaled= (np.sort(X_test_scaled, axis=0)[::-1][:slices]).T
                      y_train_scaled = (np.sort(y_train_scaled, axis=0)[::-1][:slices]).T
                      y_test_scaled = (np.sort(y_test_scaled, axis=0)[::-1][:slices]).T

                    elif method == 'DWT':

                      cA, cD = pywt.dwt(X_train_scaled.T, 'db1', axis=0)
                      X_train_scaled = np.concatenate((cA, cD), axis=0)
                      X_train_scaled = (np.sort(X_train_scaled, axis=0)[::-1][:slices]).T

                      cA, cD = pywt.dwt(X_test_scaled.T, 'db1', axis=0)
                      X_test_scaled = np.concatenate((cA, cD), axis=0)
                      X_test_scaled = (np.sort(X_test_scaled, axis=0)[::-1][:slices]).T

                      cA, cD = pywt.dwt(y_train_scaled.T, 'db1', axis=0)
                      y_train_scaled = np.concatenate((cA, cD), axis=0)
                      y_train_scaled = (np.sort(y_train_scaled, axis=0)[::-1][:slices]).T

                      cA, cD = pywt.dwt(y_test_scaled.T, 'db1', axis=0)
                      y_test_scaled = np.concatenate((cA, cD), axis=0)
                      y_test_scaled = (np.sort(y_test_scaled, axis=0)[::-1][:slices]).T
                      print('DWT=' , X_train_scaled .shape, X_test_scaled.shape, y_train_scaled.shape, y_test_scaled.shape)

                    elif method == 'PAA':

                      X_train_scaled = paa(X_train_scaled.T, n_pieces=slices)
                      X_train_scaled = X_train_scaled.T

                      X_test_scaled = paa(X_test_scaled.T, n_pieces=slices)
                      X_test_scaled = X_test_scaled.T

                      y_train_scaled = paa(y_train_scaled.T, n_pieces=slices)
                      y_train_scaled = y_train_scaled.T

                      y_test_scaled = paa( y_test_scaled.T, n_pieces=slices)
                      y_test_scaled =  y_test_scaled.T
                      print('PAA =' , X_train_scaled .shape, X_test_scaled.shape, y_train_scaled.shape, y_test_scaled.shape)


                    # Model application
                    regressor = LinearRegression()
                    regressor.fit(X_train_scaled, y_train_scaled)

                    # Generate predictions
                    y_pred_scaled = regressor.predict(X_test_scaled)

                    # Transform scaled predictions back to og scale
                    #y_pred = sc.inverse_transform(y_pred_scaled)

                    # performance (mean squared error, mean absolute error, and R-squared score)
                    mse = mean_squared_error(y_test_scaled, y_pred_scaled)
                    rmse = np.sqrt(mse)
                    #mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
                    r2 = r2_score(y_test_scaled, y_pred_scaled)

                    # Add to list of measurements
                    rmse_list.append(rmse)
                    #mae_list.append(mae)
                    r2_list.append(r2)

                #store plots
                fig = plot_confidence_interval(r2_list,method,window_size,compression,'SC') #plot function call
                pdf.savefig(fig)
                plt.close(fig)
                fig2 = plot_confidence_interval(rmse_list,method,window_size ,compression,'RMSE') #plot function call
                pdf.savefig(fig2)
                plt.close(fig2)

                # create a file name based on the method, subset index, stride, and compression
                avg_r2 = np.mean(r2_list)
                file_name = f"{method}_window size({window_size})_compression({compression} - R2 SCORES).txt"
                # write the silhouette coefficients to file
                with open(os.path.join(folder_name,stride_folder_name, compression_folder_name, file_name), 'w+') as f:
                              f.write(f"Average R2 Score: {avg_r2}\n")
                              content = str(r2_list)
                              f.write(content)
                              f.close()

                avg_RMSE = np.mean(rmse_list)
                file_name = f"{method}_window size({window_size})_compression({compression} - RMSEs).txt"
                # write the silhouette coefficients to file
                with open(os.path.join(folder_name,stride_folder_name, compression_folder_name, file_name), 'w+') as f:
                              f.write(f"Average ARI Score: {avg_RMSE}\n")
                              content = str(rmse_list)
                              f.write(content)
                              f.close()





In [ ]:
#@title Pred for window xoris 4/8/16 compression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from scipy import stats
from scipy.fft import fft, dct
import pywt

def paa(data, n_pieces):
    """
    Piecewise Aggregate Approximation (PAA) on data.
    """
    n = data.shape[0]
    piece_length = int(np.ceil(n/n_pieces))
    padded_data = np.pad(data, ((0, piece_length*n_pieces-n), (0,0)), mode='constant', constant_values=0)
    pieces = padded_data.reshape(n_pieces, piece_length, -1)
    return np.mean(pieces, axis=1)


# Load data from file
all_temperatures = np.loadtxt("/content/drive/MyDrive/temp.txt", delimiter='\t')
all_humidities = np.loadtxt("/content/drive/MyDrive/humidity.txt", delimiter='\t')
all_lights = np.loadtxt("/content/drive/MyDrive/light.txt", delimiter='\t')
all_voltages = np.loadtxt("/content/drive/MyDrive/voltage.txt", delimiter='\t')

data = all_voltages.T

# window size
window_size = 128 #@param {type:"number"}
method = 4              #@param {type:"number"} 1=DFT , 2=DCT, 3=discrete wavelet transform , 4=PAA

# lists to store metrics for each window
mse_list = []
mae_list = []
r2_list = []
subset_number = 1


# Iterate over the windows
for i in range(0, data.shape[1]-window_size*2+1, window_size):

  # Split data
  X_train, X_test, y_train, y_test = train_test_split(data[:, i:i+window_size], data[:, i+window_size:i+window_size*2], test_size=0.25, random_state=13)
  #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

  # Data scaling
  sc = MinMaxScaler()
  X_train_scaled = sc.fit_transform(X_train)
  X_test_scaled = sc.transform(X_test)
  y_train_scaled = sc.fit_transform(y_train.reshape(y_train.shape[0],-1))
  y_test_scaled = sc.transform(y_test.reshape(y_test.shape[0],-1))

  if method == 1:
        X_train_scaled = np.abs(fft(X_train_scaled, axis=0))
        X_test_scaled = np.abs(fft(X_test_scaled, axis=0))
        y_train_scaled = np.abs(fft(y_train_scaled, axis=0))
        y_test_scaled = np.abs(fft(y_test_scaled, axis=0))
        print_string = 'DFT'
  elif method == 2:
        X_train_scaled = dct(X_train_scaled, axis=0)
        X_test_scaled = dct(X_test_scaled, axis=0)
        y_train_scaled = dct(y_train_scaled, axis=0)
        y_test_scaled = dct( y_test_scaled, axis=0)
        print_string = 'DCT'
  elif method == 3:
        cA, cD = pywt.dwt(X_train_scaled, 'db1', axis=0)
        X_train_scaled = np.concatenate((cA, cD), axis=0)
        cA, cD = pywt.dwt(X_test_scaled, 'db1', axis=0)
        X_test_scaled = np.concatenate((cA, cD), axis=0)
        cA, cD = pywt.dwt(y_train_scaled, 'db1', axis=0)
        y_train_scaled = np.concatenate((cA, cD), axis=0)
        cA, cD = pywt.dwt(y_test_scaled, 'db1', axis=0)
        y_test_scaled = np.concatenate((cA, cD), axis=0)
        print_string = 'DWT'
  elif method == 4:
        X_train_scaled = paa(X_train_scaled, n_pieces=8)
        X_test_scaled = paa(X_test_scaled, n_pieces=8)
        y_train_scaled = paa(y_train_scaled, n_pieces=8)
        y_test_scaled = paa( y_test_scaled, n_pieces=8)
        print_string = 'PAA'
  else   :
      print_string = 'RAW'
  # Model application
  regressor = LinearRegression()
  regressor.fit(X_train_scaled, y_train_scaled)

  # Generate predictions
  y_pred_scaled = regressor.predict(X_test_scaled)

  # Transform scaled predictions back to og scale
  #y_pred = sc.inverse_transform(y_pred_scaled)

  # performance (mean squared error, mean absolute error, and R-squared score)
  mse = mean_squared_error(y_test_scaled, y_pred_scaled)
  mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
  r2 = r2_score(y_test_scaled, y_pred_scaled)

  print(f"Subset {subset_number} : | MSE= {mse:.5f}, | MAE= {mae:.5f}, R-squared= {r2:.5f}")
  subset_number= subset_number + 1
  # Add to list of measurements
  mse_list.append(mse)
  mae_list.append(mae)
  r2_list.append(r2)

# Saving the scores array in a text file
file_name = f"Linear Regression({window_size}) {print_string} measurements.txt"
file = open(file_name, "w+")
content = str(r2_list)
file.write(content)
file.close()

  #plot

# mean silhouette score and confidence interval
mean_score = np.mean(r2_list)
ci = stats.t.interval(0.95, len(r2_list)-1, loc=mean_score, scale=stats.sem(r2_list))

# 2D array with the lower and upper bounds of the confidence interval
yerr = np.array([[mean_score-ci[0]], [ci[1]-mean_score]])

# Plot the data with error bars
fig, ax = plt.subplots()
ax.plot(r2_list, marker='o')
ax.errorbar(x=range(len(r2_list)), y=r2_list, yerr=yerr, fmt='none', ecolor='r')
ax.set_xlabel('Subset of features')
ax.set_ylabel('R2 score')
ax.set_title('R2 scores with 95% confidence interval')
plt.show()

print(f"95% confidence interval: [{ci[0]:.5f}, {ci[1]:.5f}]")



In [ ]:
#@title Pred mono gia 1 temp
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt


# Load data from file
all_temperatures = np.loadtxt("/content/drive/MyDrive/temp.txt", delimiter='\t')
#all_humidities = np.loadtxt("/content/drive/MyDrive/humidity.txt", delimiter='\t')
#all_lights = np.loadtxt("/content/drive/MyDrive/light.txt", delimiter='\t')
#all_voltages = np.loadtxt("/content/drive/MyDrive/voltage.txt", delimiter='\t')

data = all_temperatures.T

# window size
window_size = 16 #param {type:"number"}

# lists to store metrics for each window
mse_list = []
mae_list = []
r2_list = []
subset_number = 1

# Iterate over the windows
for i in range(0, data.shape[1]-window_size+1, window_size):

  # Split data
  X_train, X_test, y_train, y_test = train_test_split(data[:, i:i+window_size-1], data[:, i+window_size], test_size=0.25, random_state=13)
  print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
  # Data scaling
  sc = MinMaxScaler()
  X_train_scaled = sc.fit_transform(X_train)
  X_test_scaled = sc.transform(X_test)
  y_train_scaled = sc.fit_transform(y_train.reshape(y_train.shape[0],1))
  y_test_scaled = sc.transform(y_test.reshape(y_test.shape[0],1))

  # Model application
  regressor = LinearRegression()
  regressor.fit(X_train_scaled, y_train_scaled)

  # Generate predictions
  y_pred_scaled = regressor.predict(X_test_scaled)

  # Transform scaled predictions back to og scale
  y_pred = sc.inverse_transform(y_pred_scaled)

  # performance (mean squared error, mean absolute error, and R-squared score)
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  print(f"Subset {subset_number} : | MSE= {mse:.5f}, | MAE= {mae:.5f}, R-squared= {r2:.5f}")
  subset_number= subset_number + 1
  # Add to list of measurements
  mse_list.append(mse)
  mae_list.append(mae)
  r2_list.append(r2)



In [ ]:
#@title Linear for entire fle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Load data from file

all_temperatures = np.loadtxt("/content/drive/MyDrive/temp.txt", delimiter='\t')
#all_humidities = np.loadtxt("/content/drive/MyDrive/humidity.txt", delimiter='\t')
#all_lights = np.loadtxt("/content/drive/MyDrive/light.txt", delimiter='\t')
#all_voltages = np.loadtxt("/content/drive/MyDrive/voltage.txt", delimiter='\t')

data = all_temperatures.T

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data [:,:-1], data[:,-1], test_size=0.25, random_state=13)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Data scaling
sc = MinMaxScaler()
X_train_scaled = sc.fit_transform (X_train)
X_test_scaled = sc.transform (X_test)

y_train_scaled = sc.fit_transform (y_train.reshape(y_train.shape[0],1))
y_test_scaled = sc.transform (y_test.reshape(y_test.shape[0],1))

# Model application
reg_line = LinearRegression()
reg_line.fit(X_train_scaled, y_train_scaled)

# Generate predictions
y_pred_scaled = reg_line.predict(X_test_scaled)

# Transform scaled predictions back to og scale
y_pred = sc.inverse_transform(y_pred_scaled)

# performance (mean squared error, mean absolute error, and R-squared score)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE: ", mse)
print("MAE: ", mae)
print("R-squared: ", r2)



# plot the predicted values against the actual values
plt.scatter(y_test, y_pred)
plt.xlabel("Actual values")
plt.ylabel("Predicted values")
plt.title("Linear Regression Results")
plt.show()
